# Figures for EPJ paper

In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from maps import rect_polygon
from paths import path_results, path_prace, path_data
from regions import SST_index_bounds
from filters import lowpass

## 1. making SMV regression plot

adjusted `SST_index_generation.py`, then executed `python SST_index_generation.py had SMV`

## 2. redoing regression plots for AMV, PMV

In [ ]:
lats, lons = xa.latitude, xa.longitude; lons, lats = np.meshgrid(lons, lats)
for i, idx in enumerate(['AMO', 'PMV', 'SMV']):
#     if i>0:  continue
    maxv = .22
    clon = [-60,220,-60][i]
#     ext = ['max', 'both', 'max'][i]
    if idx in ['AMO', 'SMV']:
        std = lowpass(xr.open_dataarray(f'{path_prace}/SST/{idx}_ds_dt_raw_{run}.nc'), 12*13).std().values
        xa = xr.open_dataset(f'{path_prace}/SST/{idx}_regr_{run}.nc')
    else:
        std = lowpass(xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_{run}.nc').pcs.isel(mode=0), 12*13).std().values
        xa = xr.open_dataset(f'{path_prace}/SST/PMV_20N_regr_{run}.nc')
    print(std)
    rect = rect_polygon(SST_index_bounds(idx))
    
    f = plt.figure(figsize=(6.4,4))
    f.text(.007, .94, 'b)', fontsize=14)
    ax = f.add_subplot(211, projection=ccrs.Robinson(central_longitude=clon))
    ax.set_global()
    ax.set_position([.01,.19,.98,.8])
    im = ax.pcolormesh(lons, lats, xa.slope*std, cmap='cmo.balance',
#     im = ax.pcolormesh(lons, lats, xa.slope, cmap='cmo.balance',
                           vmin=-maxv, vmax=maxv, transform=ccrs.PlateCarree())
    plt.tricontour(lons.flatten(), lats.flatten(), xa.pval.where(np.isnan(xa.pval.values)==False, .5).values.flatten(),
                       levels=[.01,.99], colors='purple', linestyles='dashed', transform=ccrs.PlateCarree())

    ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='grey')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])

    ax.add_patch(matplotlib.patches.Polygon(xy=rect,
                                  facecolor='none', edgecolor='k',
                                  linewidth=1, zorder=3,
                                  transform=ccrs.PlateCarree(), ), )

    cax = f.add_subplot(212)
    cax.set_position([.23,.12,.54,.05])
    label = f'HadISST regression on {["AMV","PMV","SMV"][i]} index [K/std]'
    cbar = plt.colorbar(im, cax=cax, label=label, shrink=.9, pad=.0, orientation='horizontal', extend='both', ticks=ticks)
    plt.savefig(f'{path_results}/EPJ/{["AMV","PMV","SMV"][i]}_regression_map', dpi=300)


In [ ]:
for i, idx in enumerate(['AMO', 'PMV', 'SMV']):
    time = np.arange(1870+1/24, 2019, 1/12)
    f = plt.figure(figsize=(6.4,2.5), constrained_layout=True)
    f.text(.007, .91, 'a)', fontsize=14)
    plt.axhline(0, c='r', ls=':', lw=.5)
    if idx in ['AMO', 'SOM', 'TPI', 'SMV']:
        index = xr.open_dataarray(f'{path_prace}/SST/{idx}_ds_dt_raw_{run}{ts}.nc',
                                  decode_times=False)
    elif idx=='PMV':
        index = xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_{run}{ts}.nc',
                                decode_times=False).pcs.isel(mode=0).squeeze()
#     plt.plot(time, index, c='green', lw=.5)
    plt.plot(time, index.rolling(time=12).mean(), c='grey', lw=.5)
    plt.plot(time[7*12:-7*12], lowpass(index,12*13)[7*12:-7*12])
    plt.ylabel(['AMV  [K]', 'PMV  [std]', 'SMV  [K]'][i])
    plt.xlabel('time [year C.E.]')
    plt.savefig(f'{path_results}/EPJ/{["AMV","PMV","SMV"][i]}_timeseries.eps')

## 3. GMST detrending, filtering, and HadISST regression on GMST

In [ ]:
# ds = xr.open_dataset(f'{path_data}/HadCRUT/HadCRUT.4.6.0.0.median.nc')
# spatially expliit HadCRUT data contains many missing values

In [ ]:
hadcrut = xr.open_dataarray(f'{path_data}/HadCRUT/ihad4_krig_v2_0-360E_-90-90N_n_mean1_anom_30.nc', decode_times=False) # first year 1851
GMST_had = xr.open_dataarray(f'{path_prace}/GMST/GMST_dt_yrly_had.nc')
CMIP5all = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_all.nc', decode_times=False)
CMIP5nat = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_natural.nc', decode_times=False)
CMIP5ant = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_anthro.nc', decode_times=False)

In [ ]:
time = np.arange(1870,2019)
time_ = time[7:-7]
f, ax = plt.subplots(2,1, figsize=(6,4.5), sharex=True, constrained_layout=True)
f.text(.007, .96, 'a)', fontsize=14)
f.text(.007, .5, 'b)', fontsize=14)
ax[0].plot(time, hadcrut[20:], label='HadCRUT4')
# ax[0].plot(time, CMIP5all)
ax[0].plot(time, CMIP5nat-.5, ls='--', label='CMIP5 natural')
ax[0].plot(time, CMIP5ant+.5, ls='--', label='CMIP5 anthropogenic')
ax[0].set_ylabel('GMST anomaly [K]')
ax[0].legend(frameon=False)

ax[1].axhline(0, c='r', ls=':', lw=.5)
ax[1].plot(time, GMST_had[9:], lw=.5, c='grey')
ax[1].plot(time_, lowpass(GMST_had,13)[16:-7])
ax[1].set_ylabel('detrended GMST [K]')
ax[1].set_xlabel('time [year C.E.]')

plt.savefig(f'{path_results}/EPJ/GMST_timeseries.eps')

In [ ]:
SST_had = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_tfdt_had.nc')

In [ ]:
SST_had

In [ ]:
GMST_had[9:].assign_coords(time=SST_had.time)

In [ ]:
from bd_analysis_indices import AnalyzeIndex as AI

In [ ]:
fn = f'{path_prace}/SST/GMST_yrly_tfdt_regr_had.nc'
ac = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_autocorrelation_had.nc')
AI().calculate_regression(SST_dt=SST_had, index=GMST_had[9:].assign_coords(time=SST_had.time), tavg='yrly', fn_out=fn, autocorr=ac)

In [ ]:
had_regr = xr.open_dataset(fn)

In [ ]:
had_regr.slope.plot()

In [ ]:
had_regr.pval.plot()

In [ ]:
maxv = .22
xa = xr.open_dataset(f'{path_prace}/SST/GMST_yrly_tfdt_regr_had.nc')
std = lowpass(GMST_had, 13).std().values
print(std)

f = plt.figure(figsize=(6.4,4))
f.text(.007, .94, 'c)', fontsize=14)
ax = f.add_subplot(211, projection=ccrs.Robinson(central_longitude=200))
ax.set_global()
ax.set_position([.01,.19,.98,.8])
im = ax.pcolormesh(lons, lats, xa.slope*std, cmap='cmo.balance',
                       vmin=-maxv, vmax=maxv, transform=ccrs.PlateCarree())
plt.tricontour(lons.flatten(), lats.flatten(), xa.pval.where(np.isnan(xa.pval.values)==False, .5).values.flatten(),
                   levels=[.01,.99], colors='purple', linestyles='dashed', transform=ccrs.PlateCarree())

ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='grey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])

cax = f.add_subplot(212)
cax.set_position([.23,.12,.54,.05])
label = f'HadISST regression on detrended & filtered GMST [K/std]'
cbar = plt.colorbar(im, cax=cax, label=label, shrink=.9, pad=.0, orientation='horizontal', extend='both', ticks=ticks)
plt.savefig(f'{path_results}/EPJ/GMST_regression_map', dpi=300)